In [1]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
# for neural net
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data = pd.read_csv("data/combined_expression.csv")
data.head()
data.shape

(642, 16383)

In [3]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [4]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['classification'].values

# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)

# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# Gridsearch for Input and Output Layer

In [9]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.25), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.125), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model)
epochs = [50, 75, 100, 150]
batches = [16, 32, 64, 128]
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adam', 'Adamax']
init = ['glorot_uniform', 'normal', 'uniform']
param_grid = dict(epochs=epochs, batch_size=batches,optimizer=optimizers,init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 240 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.3min


In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
grid_result.cv_results_

In [ ]:
y_pred = grid.predict(X_test)

# Testing the Model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'],optimizer=grid_result.best_params_['optimizer'],init=grid_result.best_params_['init'])
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
model.fit(X_train, y_train)
test_predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, test_predictions))
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test,test_predictions))

In [ ]:
plot_confusion_matrix(model, y_test, test_predictions)

In [40]:
model.model.save('model_1.h5')